In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments

In [5]:
from datasets import load_dataset_builder
ynat_builder = load_dataset_builder('klue', 'ynat')

In [7]:
ynat_builder.info.features

{'guid': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'label': ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None),
 'url': Value(dtype='string', id=None),
 'date': Value(dtype='string', id=None)}

In [8]:
ynat_builder.download_and_prepare('./klue_ynat/',file_format='parquet')

Generating train split:   0%|          | 0/45678 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/9107 [00:00<?, ? examples/s]

In [9]:
import pandas as pd

# Parquet 파일 경로
parquet_file = './klue_ynat/klue-train.parquet'
parquet_file2 = './klue_ynat/klue-validation.parquet'
# CSV 파일 경로 (저장될 경로)
csv_file = './klue_ynat/klue_ynat_train.csv'
csv_file2 = './klue_ynat/klue_ynat_valid.csv'
# Parquet 파일을 .Frame으로 읽기
df = pd.read_parquet(parquet_file)
df2 = pd.read_parquet(parquet_file2)

# dataFrame을 CSV 파일로 저장
df.to_csv(csv_file, index=False)
df2.to_csv(csv_file2, index=False)

In [10]:
concatenated_df = pd.concat([df, df2])

concatenated_df.to_csv('./klue_ynat/klue_ynat.csv', index=False)

In [11]:
length_df = len(df)
length_df2 = len(df2)

length_df, length_df2


(45678, 9107)

In [12]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [13]:
length_train_df = len(train_df)
length_test_df = len(test_df)

length_train_df, length_test_df

(7000, 47785)

In [20]:
45678+ 9107

54785

In [21]:
47785 + 7000

54785

In [14]:
import numpy as np
from tqdm import tqdm

In [15]:
if 'answer' not in test_df.columns:
    test_df['answer'] = np.nan
for i in tqdm(range(length_test_df)):
    for j in range(len(concatenated_df)):
        if test_df['text'].iloc[i] == concatenated_df['title'].iloc[j]:
             test_df.loc[i, 'answer'] = concatenated_df['label'].iloc[j]


  0%|          | 22/47785 [00:24<14:37:35,  1.10s/it]


KeyboardInterrupt: 

In [16]:
if 'realtext' not in test_df.columns:
    test_df['realtext'] = np.nan

In [17]:
title_to_label_dict = concatenated_df.set_index('title')['label'].to_dict()

# Map the 'text' column of test_df to this dictionary
# .get(text, np.nan) ensures that if text is not found in the dictionary, NaN is returned
test_df['answer'] = test_df['text'].map(lambda text: title_to_label_dict.get(text, np.nan))

# You can also use progress_apply from tqdm if you want a progress bar, but it's typically not needed with this efficient method
# from tqdm.auto import tqdm
# tqdm.pandas()
# test_df['answer'] = test_df['text'].progress_map(lambda text: title_to_label_dict.get(text, np.nan))





In [18]:
url_to_title_dict = concatenated_df.set_index('url')['title'].to_dict()

test_df['realtext'] = test_df['url'].map(lambda url: url_to_title_dict.get(url, np.nan))

In [19]:
test_df.head()


,ID,text,url,date,answer,realtext
0,ynat-v1_dev_00000,제임스 부상 레이커스 성탄매치서 골든스테이트에 완승종합,https://sports.news.naver.com/news.nhn?oid=001...,2018.12.26 15:16,5.0,제임스 부상 레이커스 성탄매치서 골든스테이트에 완승종합
1,ynat-v1_dev_00001,프랑스 극우정치인 르펜 노란 조끼 덕에 승승장구,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.17. 오후 8:04,4.0,프랑스 극우정치인 르펜 노란 조끼 덕에 승승장구
2,ynat-v1_dev_00002,대통령개헌안 ⑥토지공개념 명시…개발이익환수·부동산과세 강화 전망종합,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.03.21. 오후 1:09,6.0,대통령개헌안 ⑥토지공개념 명시…개발이익환수·부동산과세 강화 전망종합
3,ynat-v1_dev_00003,의사 살해 환자 경찰서 나와 법원으로,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.02. 오후 2:47,2.0,의사 살해 환자 경찰서 나와 법원으로
4,ynat-v1_dev_00004,이란 최고지도자 유럽 맹비난…핵합의 미이행 뻔뻔하고 오만,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.16. 오후 8:11,4.0,이란 최고지도자 유럽 맹비난…핵합의 미이행 뻔뻔하고 오만


In [20]:
nan_values = test_df[test_df['answer'].isna()]
print(f"{len(nan_values)}")



4616


In [21]:
nan_values.head()

,ID,text,url,date,answer,realtext
16,ynat-v1_dev_00016,미당 문체의 진미 서정주 산문 전짐 네권 발간,https://news.naver.com/main/read.nhn?mode=LS2D...,2017.03.18. 오후 3:00,NaN,미당 문체의 진미 서정주 산문 전집 4권 발간
18,ynat-v1_dev_00018,오영영호골 러은 메시 팔꿈치 가겨게 멍들고 이도 빠저...,https://sports.news.naver.com/news.nhn?oid=001...,2017.04.24 23:52,NaN,500호골 넣은 메시 팔꿈치 가격에 멍들고 이도 빠져...
21,ynat-v1_dev_00021,게시파 니중근 부영그루 푀장 용산복찌재다네 이러권 기부,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.08. 오전 10:30,NaN,게시판 이중근 부영그룹 회장 용산복지재단에 1억원 기부
31,ynat-v1_dev_00031,남자배구 대표팀 시보이 라시아선수권 소집…임동혁·진성태 함뉴,https://sports.news.naver.com/news.nhn?oid=001...,2019.08.14 13:37,NaN,남자배구 대표팀 15일 아시아선수권 소집…임동혁·진성태 합류
33,ynat-v1_dev_00033,또 어리니들부터…열두살 쿠르드 소년 죽꼬 여동생은 다리 절단,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.10.11. 오후 4:25,NaN,또 어린이들부터…12살 쿠르드 소년 죽고 여동생은 다리 절단


In [22]:
nan_values[['ID','text','realtext','url','date']].to_csv('./nan_values.csv', index=False)


In [26]:
test_df.head()

,ID,text,url,date,answer,realtext
0,ynat-v1_dev_00000,제임스 부상 레이커스 성탄매치서 골든스테이트에 완승종합,https://sports.news.naver.com/news.nhn?oid=001...,2018.12.26 15:16,5.0,제임스 부상 레이커스 성탄매치서 골든스테이트에 완승종합
1,ynat-v1_dev_00001,프랑스 극우정치인 르펜 노란 조끼 덕에 승승장구,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.17. 오후 8:04,4.0,프랑스 극우정치인 르펜 노란 조끼 덕에 승승장구
2,ynat-v1_dev_00002,대통령개헌안 ⑥토지공개념 명시…개발이익환수·부동산과세 강화 전망종합,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.03.21. 오후 1:09,6.0,대통령개헌안 ⑥토지공개념 명시…개발이익환수·부동산과세 강화 전망종합
3,ynat-v1_dev_00003,의사 살해 환자 경찰서 나와 법원으로,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.02. 오후 2:47,2.0,의사 살해 환자 경찰서 나와 법원으로
4,ynat-v1_dev_00004,이란 최고지도자 유럽 맹비난…핵합의 미이행 뻔뻔하고 오만,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.16. 오후 8:11,4.0,이란 최고지도자 유럽 맹비난…핵합의 미이행 뻔뻔하고 오만


In [25]:
test_df['answer'] = test_df['answer'].astype(int)
test_df[['ID', 'answer', 'text', 'url']].to_csv('answer.csv', index=False)


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [27]:
title_to_label_dict = concatenated_df.set_index('url')['label'].to_dict()

# Map the 'text' column of test_df to this dictionary
# .get(text, np.nan) ensures that if text is not found in the dictionary, NaN is returned
test_df['answer'] = test_df['url'].map(lambda text: title_to_label_dict.get(text, np.nan))

# You can also use progress_apply from tqdm if you want a progress bar, but it's typically not needed with this efficient method
# from tqdm.auto import tqdm
# tqdm.pandas()
# test_df['answer'] = test_df['text'].progress_map(lambda text: title_to_label_dict.get(text, np.nan))





In [28]:
test_df.head()

,ID,text,url,date,answer,realtext
0,ynat-v1_dev_00000,제임스 부상 레이커스 성탄매치서 골든스테이트에 완승종합,https://sports.news.naver.com/news.nhn?oid=001...,2018.12.26 15:16,5,제임스 부상 레이커스 성탄매치서 골든스테이트에 완승종합
1,ynat-v1_dev_00001,프랑스 극우정치인 르펜 노란 조끼 덕에 승승장구,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.17. 오후 8:04,4,프랑스 극우정치인 르펜 노란 조끼 덕에 승승장구
2,ynat-v1_dev_00002,대통령개헌안 ⑥토지공개념 명시…개발이익환수·부동산과세 강화 전망종합,https://news.naver.com/main/read.nhn?mode=LS2D...,2018.03.21. 오후 1:09,6,대통령개헌안 ⑥토지공개념 명시…개발이익환수·부동산과세 강화 전망종합
3,ynat-v1_dev_00003,의사 살해 환자 경찰서 나와 법원으로,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.01.02. 오후 2:47,2,의사 살해 환자 경찰서 나와 법원으로
4,ynat-v1_dev_00004,이란 최고지도자 유럽 맹비난…핵합의 미이행 뻔뻔하고 오만,https://news.naver.com/main/read.nhn?mode=LS2D...,2019.07.16. 오후 8:11,4,이란 최고지도자 유럽 맹비난…핵합의 미이행 뻔뻔하고 오만


In [29]:
test_df[['ID', 'answer', 'text', 'url']].to_csv('answer.csv', index=False)


: 